In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# !pip install gluoncv

In [3]:
import sys

sys.path.append('~/SageMaker/AmazonSageMaker-defect-classifier-poca/patch_classification')

In [4]:
import os
import random

import pandas as pd

import PIL
from PIL import ImageDraw

from IPython.display import Image, display
from IPython.html.widgets import interact

# from preprocessing import get_fname

# import utils

/home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [5]:
def get_orig_patch_images(df, idx, version, camera_type, dataset_type):
    
    img_dir = get_img_dir(version, camera_type, dataset_type)
    original_img_dir = get_origin_img_dir(camera_type)
    
    img_info = df.iloc[idx]
    
    patch_img_fname = img_info.patch_fname
    patch_xy = [img_info.patch_x0, img_info.patch_y0]
    orig_bbox = [img_info.RoiX0, img_info.RoiY0, img_info.RoiX1, img_info.RoiY1]
    bbox = [img_info.patch_bbox_x0, img_info.patch_bbox_y0, 
            img_info.patch_bbox_x1, img_info.patch_bbox_y1]
    # load the patch image and draw a rectangle of the defect bound box
    img_full_fname = os.path.join(img_dir, str(img_info.defect_name), patch_img_fname)
    img = PIL.Image.open(img_full_fname).convert('RGB')
    img_with_bbox = ImageDraw.Draw(img)
    img_with_bbox.rectangle(bbox, outline='#FF0000', width=2)
    img.save(tmp_img_fname)

    # load the original image and draw a rectangle of the patch
    original_img_full_fname = os.path.join(original_img_dir, str(img_info.ClassId), img_info.img_fname)
    org_img = PIL.Image.open(original_img_full_fname).convert('RGB')
    org_img_with_bbox = ImageDraw.Draw(org_img)
    patch_bbox = [patch_xy[0], patch_xy[1], patch_xy[0] + img.size[0], patch_xy[1] + img.size[1]]
    org_img_with_bbox.rectangle(patch_bbox, outline='#0000FF', width=2)
    org_img_with_bbox.rectangle(orig_bbox, outline='#FF0000', width=2)
    org_img.save(tmp_original_img_fname, 'JPEG', quality=100)
    
    net = models[f'{camera_type}-v{version}']

    y_true_label, y_pred_label = modelAnalysisUtil.get_predicted_label(net, img_full_fname, camera_type, version)

    cam_fname = modelAnalysisUtil.get_CAM_heatmap(net, img_full_fname, camera_type, version)
    
    return img_info, org_img.size, img.size, bbox, y_true_label, y_pred_label, cam_fname

def display_orig_patch_img(version, camera_type, dataset_type, index):
    
    train_df = patch_dfs[f'{camera_type}-v{version}-train']
    
    df = patch_dfs[f'{camera_type}-v{version}-{dataset_type}']
    
    img_info, org_img_size, patch_img_size, bbox_in_patch, true_label, pred_label, cam_fname = get_orig_patch_images(df, index, version, camera_type, dataset_type)
    
    
    pred_result = 'CORRECT' if true_label == pred_label else 'NOT CORRECT'

    # Find a reference image for the predicted class
    train_df_defect = train_df[train_df.defect_name==pred_label]
    num_defect_examples = len(train_df_defect)
    defect_index = random.randint(0, num_defect_examples)
    reference_patch_img_fname = train_df_defect.iloc[defect_index].patch_fname
    reference_patch_img_full_fname = os.path.join(f'{root}/dataset_v{version}/{patch_dir}', camera_type, 'train', pred_label, reference_patch_img_fname)

    model_name = models_name[f'{camera_type}-v{version}']
    
    print(f'image name           : {img_info.img_fname}')
    print(f'orignal image - size : {org_img_size}')
    print(f'orignal image - bbox : ({img_info.RoiX0}, {img_info.RoiY0}) , ({img_info.RoiX1}, {img_info.RoiY1})')
    print(f'patch image   - size : {patch_img_size}')
    print(f'patch image   - bbox : {bbox_in_patch}')
    print(f'patch ground truth   : {true_label}')
    print(f'predcited            : {pred_label} as {pred_result}')
    print(f'model name           : {model_name}')

    
    
    # make a prediction
    display(Image(cam_fname))
    display(Image(tmp_img_fname))
    display(Image(tmp_original_img_fname))
    print(f'Reference image of defect {pred_label} - {reference_patch_img_fname}')
#     print(f'Full path {reference_patch_img_full_fname}')
    display(Image(reference_patch_img_full_fname))
    
    print('-'*100)
    

In [6]:
def get_orig_image(df, idx, camera_type, version='2'):
    
    original_img_dir = get_origin_img_dir(camera_type)
    
    img_info = df.iloc[idx]
    
    img_fname = get_fname(img_info)
    orig_bbox = [img_info.RoiX0, img_info.RoiY0, img_info.RoiX1, img_info.RoiY1]
    
    # load the original image and draw a rectangle of the patch
    original_img_full_fname = os.path.join(original_img_dir, str(img_info.ClassId), img_fname)
    org_img = PIL.Image.open(original_img_full_fname).convert('RGB')
    org_img_with_bbox = ImageDraw.Draw(org_img)
    org_img_with_bbox.rectangle(orig_bbox, outline='#FF0000', width=2)
    org_img.save(tmp_original_img_fname, 'JPEG', quality=100)
    
    return img_info, org_img.size

def display_orig_img(abnormal_type, camera_type, index):
    img_info, org_img_size = get_orig_image(abnormal_dfs[f'{camera_type}-{abnormal_type}'], index, camera_type)

    img_fname = get_fname(img_info)
    print(f'image name           : {img_fname}')
    print(f'orignal image - size : {org_img_size}')
    print(f'orignal image - bbox : ({img_info.RoiX0}, {img_info.RoiY0}) , ({img_info.RoiX1}, {img_info.RoiY1})')
    print(f'orignal image - bbox : ({img_info.RoiX1 - img_info.RoiX0} x {img_info.RoiY1 - img_info.RoiY0}) (w,h)')

    display(Image(tmp_original_img_fname))
    

## Patch browser

In [7]:
def get_img_dir(version='2', camera_type='GA', dataset_type='val'):
    img_dir = f'{root}/dataset_v{version}/{patch_dir}/{camera_type}/{dataset_type}'
    return img_dir

def get_origin_img_dir(camera_type='GA'):
    original_img_dir = f'{root}/datasets/Images/{camera_type}'
    return original_img_dir

root = '/home/ec2-user/SageMaker' if 'SageMaker' in os.getcwd() else '/mlsl'

tmp_img_fname = f'{root}/tmp.jpg'
tmp_original_img_fname = f'{root}/tmp-original.jpg'

versions = ['2', '3']
camera_types = ['GA', 'GI']
dataset_types = ['train', 'val', 'test']
abnormal_types = ['large_bbox', 'small_image', 'zerobbox']

#### Change the directory name

In [8]:
# SET THIS #

patch_dir = 'datasets-patches-4'
exp_nums = ['exp26', 'exp27']

In [9]:
from model_analysis_utils import ModelAnalysisUtil

modelAnalysisUtil = ModelAnalysisUtil()

models = {}
models_name = {}

# Loading models of experiments in exp_nums
for exp_num in exp_nums:
    net, camera_type, version = modelAnalysisUtil.get_model(exp_num)
    models_name[f'{camera_type}-v{version}'] = modelAnalysisUtil.get_model_name(exp_num)
    models[f'{camera_type}-v{version}'] = net
    
for exp_num, model_name in zip(exp_nums, models_name):
    print(f'{exp_num} -> {models_name[model_name]} as {model_name}')



exp26 -> exp26-ResNet18_v2-GI-datasetv2-26-0827_124359.params as GI-v2
exp27 -> exp27-ResNet18_v2-GA-datasetv2-27-0827_135338.params as GA-v2


In [10]:
patch_dfs = {}

for camera_type in camera_types:
    for dataset_type in dataset_types:
        for version in versions:
            csv_fname = f'{root}/dataset_v{version}/{patch_dir}/patch_list_{camera_type}_{dataset_type}.csv'
            patch_dfs[f'{camera_type}-v{version}-{dataset_type}'] = pd.read_csv(csv_fname).sort_values(by=['defect_name'])
        

In [11]:
for camera_type in camera_types:
    for dataset_type in dataset_types[1:]:
        for version in versions:
            if f'{camera_type}-v{version}' in models:
                interact(display_orig_patch_img, 
                         version=[version],
                         camera_type=[camera_type], 
                         dataset_type=[dataset_type], 
                         index=(0,len(patch_dfs[f'{camera_type}-v{version}-{dataset_type}'])-1))
        

interactive(children=(Dropdown(description='version', options=('2',), value='2'), Dropdown(description='camera…

interactive(children=(Dropdown(description='version', options=('2',), value='2'), Dropdown(description='camera…

interactive(children=(Dropdown(description='version', options=('2',), value='2'), Dropdown(description='camera…

interactive(children=(Dropdown(description='version', options=('2',), value='2'), Dropdown(description='camera…